In [ ]:
!pip install --no-cache-dir -r requirements.txt
!python Create_synth_data.py

^C



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col
import polars as pl
import time
logfile = "logfiles.log"


In [ ]:
# time the code
start = time.time()
# read the first 50 lines of the log file into a DataFrame
df = pl.scan_csv(logfile, has_header=False, separator = '~')

df_eager = df.with_columns(
    pl.col("column_1").str.extract(r"((\d{1,3}\.){3}\d{1,3})").alias("IP"),
    pl.col("column_1").str.extract(r"\[(\d{1,2}/\w{3}/\d{4})").alias("date"),
    pl.col("column_1").str.extract(r'"(\w*)').alias("method"),
    pl.col("column_1").str.extract(r'"\w* ((/\w*){1,3}) HTTP/1.0').alias("requested_path"),
    pl.col("column_1").str.extract(r'"\w* [/\w*]* (\w*/\d*\.\d*)').alias("protocol"),
    pl.col("column_1").str.extract(r'1\.0" (\d{3})').alias("status"),
    pl.col("column_1").str.extract(r'1\.0" \d{3} (\d{1,10})').alias("response_size"),
    pl.col("column_1").str.extract(r'1\.0"\s\d{3}\s\d+\s"([^"]*)"').alias("Referrer URL"),
    pl.col("column_1").str.extract(r'HTTP/1\.0"\s\d{3}\s\d+\s"[^"]*"\s"([^"]+)"', group_index=1).alias("user-agent_string"),
    pl.col("column_1").str.extract(r'(\d+$)').alias("response_time"),
    pl.col("column_1").str.extract(r":(\d{2}):\d{2}:\d{2}").cast(pl.Int32).alias("hour"),
    pl.col("column_1").str.extract(r":\d{2}:(\d{2}):\d{2}").cast(pl.Int32).alias("minute"),
    pl.col("column_1").str.extract(r":\d{2}:\d{2}:(\d{2})").cast(pl.Int32).alias("second"),
    pl.col("column_1").str.extract(r"\+(\d{2})\d{2}").cast(pl.Int32).alias("timezone_offset"),
).with_columns(
    ((pl.col("hour") - pl.col("timezone_offset")) % 24).alias("adjusted_hour"),
).with_columns(
    pl.concat_str([
        pl.col("adjusted_hour").cast(pl.Utf8).str.zfill(2),
        pl.col("minute").cast(pl.Utf8).str.zfill(2),
        pl.col("second").cast(pl.Utf8).str.zfill(2),
    ], separator=":").alias("time(UTC+0)")
).select(
    "IP", "date", "time(UTC+0)", "method", "requested_path", "protocol",
    "status", "response_size", "Referrer URL", "user-agent_string", "response_time",
).collect()

#df.with_columns(pl.col("column_1").str.slice(70)).collect()
#df.with_columns(pl.col("column_1").str.extract(r'"\w* (/\w*[/, ]\w*[/, ]\w*)').alias("Method")).collect()
    
# finish time the code

# upload the DataFrame to a CSV file
df_eager.write_csv("logfiles.csv", separator="~")
end = time.time()
print("Elapsed time: ", end - start)

Elapsed time:  0.27782320976257324


In [532]:
spark = SparkSession.builder \
    .appName("LogFileAnalysis") \
    .getOrCreate()

# read csv_file with spark
logs = spark.read.csv("logfiles.csv", sep="~", header=True)
#logs.show(5, truncate=False)
tiny_log = logs.limit(50)
tiny_log.select("IP").show()

+---------------+
|             IP|
+---------------+
| 73.114.209.133|
|223.114.129.136|
|   136.219.6.68|
|    12.234.4.43|
| 199.150.229.44|
|   97.38.198.61|
|   46.255.2.194|
|   24.24.11.243|
| 133.156.244.79|
| 204.211.28.116|
| 53.133.177.217|
| 116.240.217.24|
|  70.87.173.108|
|  74.13.206.149|
|104.196.216.196|
|211.239.135.252|
| 37.163.139.237|
| 122.179.157.36|
| 202.112.219.25|
| 177.244.146.70|
+---------------+
only showing top 20 rows



In [ ]:
#logs_tiny.select(regexp_extract('value', r'(\d+\.\d+\.\d+\.\d+)', 1).alias('ip')).show(truncate=False)
logs_tiny.select(regexp_extract('IP', r'[(\d{1,2}/\n{3}/\d{4})]', 1).alias('IP')).show()

Py4JJavaError: An error occurred while calling o110.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 8) (DESKTOP-6O57JT6 executor driver): org.apache.spark.SparkRuntimeException: [INVALID_PARAMETER_VALUE.REGEX_GROUP_INDEX] The value of parameter(s) `idx` in `regexp_extract` is invalid: Expects group index between 0 and 0, but got 1.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidRegexGroupIndexError(QueryExecutionErrors.scala:357)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase$.checkGroupIndex(regexpExpressions.scala:735)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase.checkGroupIndex(regexpExpressions.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java